# This project aims to utilize Long Short Term Memory neural networks on a variable set of momentum indicators to predict stock price evolution.

## The dataset we use daily stock price data for 10 tickers, taken from the quandl python api

In [2]:
import pandas as pd
import quandl
quandl.ApiConfig.api_key = "47d6qS5DtPwi1miuHQHh"
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

## Momentum Indicators
Firstly it is necesarry to establish our predictive variables. We calculate the following momentum indicators:

- MACD:
    - Moving Average Convergence Divergence expresses the relationship between 2 moving averages of a security. We compute it by subtracting the 26 day exponential moving average (EMA) from the 12 day EMA. 
- Stochastic RSI:
    - The Relative Strength Index (RSI) describes a momentum indicator that measures the magnitude of recent price changes in order to evaluate overbought or oversold conditions in the price of a stock or other asset
    - The Stochastic RSI applies the stochastic oscillator formula to the relative strength index.
    - It indicates overbought and oversold conditions in a security's price
- Bollinger Percentage
    -Quantifies the relationship between Bollinger bands and price action to indicate overbought/oversold conditions
- Bollinger Band Width- Difference between upper and lower Bollinger bands
- Ease of Movement- Volume and momentum oscialltor which expresses the ease at which price moves determined by the volume profile


In [ ]:
def Indicators(df,period):
    #df['pct_change'] = df.open.pct_change()
    #df['log_return'] = np.log(1 + df['pct_change'])
    
    df['Middle Band'] = df['open'].rolling(window=20).mean()
    df['20 Day STD'] = df['open'].rolling(window=20).std()
    df['Upper Band']= df['Middle Band']+df['20 Day STD']*2
    df['Lower Band']= df['Middle Band']-df['20 Day STD']*2
    df['Band Width']= ((df['Upper Band'] - df['Lower Band']) / df['Middle Band']) * 100
    df['Bollinger_percent'] = ((df.close-df['Lower Band'])/(df['Upper Band']-df['Lower Band']))*100
    
    
    #df['exp1'] = df.apply(lambda x: x['close'].ewm(span=12,min_periods=12).mean())
    #df['exp2'] = df.apply(lambda x: x['close'].ewm(span=26,min_periods=26).mean())
    df['exp1'] = df.close.ewm(span=12,min_periods=12, adjust=False).mean()
    df['exp2'] = df.close.ewm(span=26,min_periods=26, adjust=False).mean()
    
    df['MACD'] = df['exp1']-df['exp2']
    
    df['exp3'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    RSI_computer = pd.DataFrame(df.close.shift().diff())
    RSI_computer.rename(columns={'close': 'delta'},inplace = True)
    
    RSI_computer['u'] = RSI_computer.delta * 0
    RSI_computer['d'] = RSI_computer.u.copy()
    
    RSI_computer['u'].loc[(RSI_computer.delta > 0)] = RSI_computer.delta.loc[(RSI_computer.delta >0)]
    RSI_computer['d'].loc[(RSI_computer.delta < 0)] = -RSI_computer.delta.loc[(RSI_computer.delta <0)]
    
    RSI_computer['u'].loc[RSI_computer.u.index[period-1]]= np.mean(RSI_computer.u.loc[:period])
    RSI_computer.u = RSI_computer.u.drop(RSI_computer.u.index[:(period-1)])
    
    RSI_computer['d'].loc[RSI_computer.d.index[period-1]]= np.mean(RSI_computer.d.loc[:period])
    RSI_computer.d = RSI_computer.d.drop(RSI_computer.d.index[:(period-1)])
    
    RSI_computer['RS'] = RSI_computer.u.ewm(span=period-1, adjust=False).mean()/ RSI_computer.d.ewm(span=period-1, adjust=False).mean()
    
    df['RSI_final']= pd.Series(100-100/(1+RSI_computer.RS))
    #df.join(RSI_final) 
    #print('Fine')
    
    df['Stochastic_Oscillator'] = (df.close-df.low.rolling(14).min()/df.high.rolling(14).max()-df.low.rolling(14).min())*100 
    #print('Fine2')
    
    dm = ((df['high'].shift() + df['low']/2) - ((df['high'].shift() + df['low'].shift())/2))
    br = (df['volume'] / 100000000) / ((df['high'] - df['low']))
    
    EVM = dm / br 
    df['EVM_MA'] = pd.Series(EVM.rolling(14).mean()) 
    
    
    df_prelim = df[['date','Bollinger_percent','MACD','exp3','RSI_final','Stochastic_Oscillator','close','EVM_MA','Band Width']]
    
    
    
    df_final = DealWithMissing(df_prelim)
    #print('Fine3')
    #df_final2 = Normalise(df_final)
    #print('Fine4')
    df_final['Difference']= df_final['close'].diff(periods=-1)
    #print('Fine5')
    df_final['UP_DOWN'] = np.where(df_final['Difference']>0,1,0) 
    #print('Fine6')
    df_final3 = DealWithMissing(df_final)

        
    
    df_submit = df_final3[['date','Bollinger_percent','MACD','exp3','RSI_final','Stochastic_Oscillator','close','EVM_MA','Band Width','UP_DOWN']]
    
            
    
    return(df_submit,df_final3)

It is often useful to standardize our variables by subtracting the variable means and dividing by the standard deviation for faster training in lieu of gradient descent.

In [ ]:
def Normalise(df):
    
    normalise_df = (df-df.mean())/df.std()
    
    return(normalise_df)

We clean the dataset by removing missing data as there is risk of adding bias with backfilling

In [ ]:
def DealWithMissing(df):
    fixed_df = df.dropna(how='any')
    return(fixed_df)

In [ ]:
We create functions to generate dicts for each ticker and  

In [ ]:
def dict_maker(df,list_tickers):
    dict1 = {}
    for i in list_tickers:
        dict1[i]=df[df['ticker']==i]
    return(dict1)


We reshape our feature set into 60 day intervals, creating a 3D matrix for our training set. Additionally we separate our target of close prices.

We construct the Long short term memory model with 4 hidden layers with 20% dropout.

In [ ]:
def LSTM_Price(df,key):
    
    #df = Indicative_dicts['AAPL']
    
    colnum=df.shape[0]
    colnum-1000

    
    sc = MinMaxScaler(feature_range = (0, 1))
    sc1 = MinMaxScaler(feature_range = (0, 1))
    
    #sc1 = MinMaxScaler(feature_range = (0, 1))

    #Real stock values for testing
    Close_prices = df[['date','close']]
    Close_prices= Close_prices.iloc[colnum-1000:,:]
    #Close_prices = Close_prices.reshape(-1,1)
    #Close_prices = sc1.fit_transform(Values_price)
    
    #f1=Close_prices[['close']]
    #f1.reshape(-1,1)
    
    #scaled features
    Features = df.iloc[:,1:9].values
    
    f1 = Features[:,5]
    f1=f1.reshape(-1,1)
    #Placeheld = sc1.fit_transform(f1)
    Features_Scaled = sc.fit_transform(Features)
    #Values_c = ADI_df.iloc[:,9].values
    
        
    X_train = []
    y_train = []
    
    for i in range(60,colnum-1000):
        X_train.append(Features_Scaled[i-60:i,:])
        y_train.append(Features_Scaled[i,5])
        
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    X_test = []
    y_test = []
    
    for i in range(colnum-1000,colnum):
        X_test.append(Features_Scaled[i-60:i,:])
        y_test.append(Features_Scaled[i,5])
        
    X_test, y_test = np.array(X_test), np.array(y_test)    
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 8))



        
    regressor = Sequential()    
    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 8)))
    regressor.add(Dropout(0.2))
    
    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    
    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    
    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    
    # Adding the output layer
    regressor.add(Dense(units = 1))
    
    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, epochs = 15, batch_size = 32,validation_split=0.2,verbose=0)


        
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc1.inverse_transform(predicted_stock_price)
    predicted_stock_price = pd.DataFrame(predicted_stock_price)
    
   # together = pd.concat([Close_prices, pd.DataFrame(predicted_stock_price)], axis=1, ignore_index=True)
    together = pd.concat([Close_prices.reset_index(drop=True), predicted_stock_price], axis=1)    



Now we load the data for 10 tickers and train our models for each ticker.

In [ ]:
tickers_10_data=pd.read_csv('./tickers.csv')
list_tickers=tickers_10_data['Ticker'].tolist()


_10_tickers_data = pd.read_csv("./Data_main.csv") 


_10_tickers_data = _10_tickers_data.iloc[::-1]

if(_10_tickers_data.isnull().values.any()):
    Tickers_ready = DealWithMissing(_10_tickers_data)
else:
    Tickers_ready=_10_tickers_data 

Ticks_dict = dict_maker(Tickers_ready,list_tickers)

Indicative_dicts = {}
Feature_eng_dicts = {}

for key in Ticks_dict:
    Indicative_dicts[key],Feature_eng_dicts[key]=Indicators(Ticks_dict[key],14)

for key in Indicative_dicts:
    LSTM_Price(Indicative_dicts[key],key)

Let us observe the out of sample performance for our 10 tickers. Note we test on the dates consequently following our training dates to utilize the sequential retention of the neural LSTMs.

<img src="Plots/AAPL-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/ADI-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/CNP-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/DLTR-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/FLS-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/HAS-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/PBCT-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/RHI-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/TGT-1.png" alt="Drawing" style="width: 600px;"/>

<img src="Plots/WBA-1.png" alt="Drawing" style="width: 600px;"/>